In [191]:
import pandas as pd
#* have average top 4 odds 
#must place 1st, 2nd, 11th, or 12th 40% of the time 
#and average a finish +1 seconds from the median race time

In [192]:
#storing the three datasets
race_data = pd.read_csv('race_db.csv')
horse_data = pd.read_csv('horse_db.csv')
horse_race_data = pd.read_csv('races with distance')
speed_stats = pd.read_csv('speed-stats.csv')

In [193]:
#observing the data
race_data.head()


,race_id,start_time,name,distance,class,country,city,weather,fee,prize_pool_first,prize_pool_second,prize_pool_third
0,5fO3jsyv,2021-06-15 23:52:22,Hallmark Grand Prix,1400,4,Australia,Melbourne,Sunny,0.002,0.01296,0.00540,0.00324
1,9H9Rkdoo,2021-06-15 23:47:22,Porlamar Dash,1200,5,Venezuela,Porlamar,Sunny,0.001,0.00648,0.00270,0.00162
2,ljonwI7K,2021-06-15 23:42:22,Brazillian Oaks,2400,4,Brazil,Sao Paulo,Sunny,0.002,0.01296,0.00540,0.00324
3,BEabgEVC,2021-06-15 23:39:00,Johor Bahru Challenge,1400,5,Malaysia,Johor Bahru,Sunny,0.000,0.00060,0.00025,0.00015
4,K6xYJYAl,2021-06-15 23:35:22,San Jose Dirt Mile,1600,5,United States,San Jose,Sunny,0.001,0.00648,0.00270,0.00162


In [194]:
horse_data.head()

,horse_id,name,genotype,bloodline,breed_type,color,super_coat,mother,father,horse_type,birthday
0,0,Thin Frivolity,Z1,Nakamoto,genesis,Hairy Canary,f,NaN,NaN,Colt,2019-04-04 01:39:07
1,1,Chase Jackson,Z1,Nakamoto,genesis,Cornsilk,f,NaN,NaN,Mare,2019-01-17 00:21:42
2,2,Wall Street,Z1,Nakamoto,genesis,Bittersweet Shimmer,f,NaN,NaN,Stallion,2019-01-17 00:32:57
3,3,Millions,Z1,Nakamoto,genesis,Fandango,f,NaN,NaN,Filly,2019-01-17 02:44:23
4,4,Guns and Horses,Z1,Nakamoto,genesis,White Smoke,f,NaN,NaN,Colt,2019-01-18 06:44:24


In [195]:
horse_race_data.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance
0,sa3B6YKZ,2838,15.78,10.0,58.5152,2021-06-01,1000
1,sa3B6YKZ,6207,10.47,3.0,58.3062,2021-06-01,1000
2,sa3B6YKZ,7914,9.40,4.0,58.3068,2021-06-01,1000
3,sa3B6YKZ,8866,9.70,8.0,58.4155,2021-06-01,1000
4,sa3B6YKZ,9184,25.55,11.0,58.5599,2021-06-01,1000


In [196]:
#taking records with only 1800 distance, output shows number of records left
horse_race_data100k = horse_race_data[horse_race_data['distance'] == 1800]
#using subset of above data with only odds and race_id for speed purposes
oddsRacesDf = horse_race_data100k[['race_id','odds']] 
horse_race_data100k.shape[0] 

64664

In [197]:

oddsRacesDf.head()

,race_id,odds
96,VH5mOjX8,14.52
97,VH5mOjX8,12.32
98,VH5mOjX8,10.51
99,VH5mOjX8,11.46
100,VH5mOjX8,11.67


In [198]:
#function that takes in the raceId and returns the sorted odds for that race
def OddSort(raceId):
    df = oddsRacesDf[oddsRacesDf['race_id'] == raceId] #storing the sliced Df of the race_ids records in a new df
    sortOdds = df['odds'].sort_values() #storing the odds of the race in a series, and sorting
    return sortOdds #return the sorted odds

In [199]:
#testing the program
sortOdds = OddSort('VH5mOjX8') 

In [200]:
#output of test: passed
sortOdds

104     8.82
98     10.51
103    11.05
102    11.20
101    11.41
99     11.46
100    11.67
105    11.84
97     12.32
96     14.52
107    15.15
106    19.77
Name: odds, dtype: float64

In [201]:

stored_Id = '000' #variable to store each race_id in the loop, will be passed into the sort function on each iteration
fourthPlaceOdds = [] #list to store fourth place odds
raceList = [] #list to store each race to ensure that a races odds are not used twice
raceIdMap = {} #dictionary to create keys for each id and then store associated odds
for race_Id in oddsRacesDf['race_id']: #loop through each id in series
    if race_Id != stored_Id and race_Id not in raceList: #if the race_id is not the store_id and race is not in racelist
        stored_Id = race_Id
        raceList.append(race_Id) #append the race id to the race list 
        seriesOdds = OddSort(stored_Id) #calling the sorted odds function and storing the resulting sorted series
        if len(seriesOdds.tolist()) < 4: #this if block is for races that did not have less than 4 records
            fourthPlaceOdds.append(seriesOdds.tolist()[-1])
        else:
            fourthPlaceOdds.append(seriesOdds.tolist()[3]) #taking the fourth odd in the sorted series
            
        
        

In [202]:
#observing if fourth place odds of each race were stored
fourthPlaceOdds[:12]

[11.2, 10.0, 8.76, 8.76, 10.7, 11.0, 10.17, 10.66, 10.4, 10.41, 10.86, 10.63]

In [203]:
#populating the dictionary with the race as key and fourthplaceodds list as value
for i in range(len(raceList)):
    raceIdMap[raceList[i]] = fourthPlaceOdds[i]

In [204]:
#adding a column to table to put the fourth place odds for each race
horse_race_data100k['Fourth Place Odds'] = ""

<ipython-input-204-3974cbe8a049>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_race_data100k['Fourth Place Odds'] = ""


In [205]:
horse_race_data100k.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds
96,VH5mOjX8,7248,14.52,11.0,109.1121,2021-06-01,1800,
97,VH5mOjX8,8313,12.32,3.0,108.2773,2021-06-01,1800,
98,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,
99,VH5mOjX8,10888,11.46,1.0,108.1116,2021-06-01,1800,
100,VH5mOjX8,17070,11.67,4.0,108.2783,2021-06-01,1800,


In [206]:
horse_race_data100k = horse_race_data100k.reset_index() #resetting index for looping purposes
horse_race_data100k = horse_race_data100k.drop(['index'], axis = 1)
horse_race_data100k.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds
0,VH5mOjX8,7248,14.52,11.0,109.1121,2021-06-01,1800,
1,VH5mOjX8,8313,12.32,3.0,108.2773,2021-06-01,1800,
2,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,
3,VH5mOjX8,10888,11.46,1.0,108.1116,2021-06-01,1800,
4,VH5mOjX8,17070,11.67,4.0,108.2783,2021-06-01,1800,


In [207]:
#looping through data and populating the empty column with the fourth place odds from the dictionary
for row in range(horse_race_data100k.shape[0]):
    horse_race_data100k.loc[row, 'Fourth Place Odds'] = raceIdMap[horse_race_data100k.loc[row,'race_id']]

In [208]:
#checking to see if odds populated
horse_race_data100k.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds
0,VH5mOjX8,7248,14.52,11.0,109.1121,2021-06-01,1800,11.2
1,VH5mOjX8,8313,12.32,3.0,108.2773,2021-06-01,1800,11.2
2,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2
3,VH5mOjX8,10888,11.46,1.0,108.1116,2021-06-01,1800,11.2
4,VH5mOjX8,17070,11.67,4.0,108.2783,2021-06-01,1800,11.2


In [209]:
#finding the average fourth place odd
averageFourthPlaceOdds = sum(fourthPlaceOdds)/len(fourthPlaceOdds)
print(averageFourthPlaceOdds)

10.026459454444243


In [210]:
#now looking to create a dictionary with each horse as a key and a list of its race times mapped to it
horseOddsMap = {}
for row in range(horse_race_data100k.shape[0]):
    if str(horse_race_data100k.loc[row,'horse_id']) in horseOddsMap:
        horseOddsMap[str(horse_race_data100k.loc[row,'horse_id'])].append(horse_race_data100k.loc[row,'odds'])
    else:
        horseOddsMap[str(horse_race_data100k.loc[row,'horse_id'])] = [horse_race_data100k.loc[row,'odds']]

In [211]:
#now taking the average of each horses list from the horseOddsDictionary and populating it in new dictionary
horseOddsAverageMap = {}
for key,val in horseOddsMap.items():
    if key not in horseOddsAverageMap:
        horseOddsAverageMap[key] = sum(val)/len(val)

In [212]:
#result of last loop run
horseOddsAverageMap['7248']

13.525500000000003

In [213]:
#adding in a column to store the average odds for the horses
horse_race_data100k['Average Odds'] = ""
horse_race_data100k.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds
0,VH5mOjX8,7248,14.52,11.0,109.1121,2021-06-01,1800,11.2,
1,VH5mOjX8,8313,12.32,3.0,108.2773,2021-06-01,1800,11.2,
2,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2,
3,VH5mOjX8,10888,11.46,1.0,108.1116,2021-06-01,1800,11.2,
4,VH5mOjX8,17070,11.67,4.0,108.2783,2021-06-01,1800,11.2,


In [214]:
#populating the average odds columns
for row in range(horse_race_data100k.shape[0]):
    horse_race_data100k.loc[row,'Average Odds'] = horseOddsAverageMap[str(horse_race_data100k.loc[row,'horse_id'])]

In [215]:
#average odds successfully populated
horse_race_data100k.head()


,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds
0,VH5mOjX8,7248,14.52,11.0,109.1121,2021-06-01,1800,11.2,13.5255
1,VH5mOjX8,8313,12.32,3.0,108.2773,2021-06-01,1800,11.2,11.4975
2,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2,9.679583
3,VH5mOjX8,10888,11.46,1.0,108.1116,2021-06-01,1800,11.2,10.654545
4,VH5mOjX8,17070,11.67,4.0,108.2783,2021-06-01,1800,11.2,11.975


In [216]:
averageFourthPlaceOdds = horse_race_data100k['Fourth Place Odds'].mean() #taking average fourth place odds 
averageFourthPlaceOdds

10.02641701719625

In [217]:
#creating a new dataframe where the average odds column is less than or equal to the average fourth place odds
fourthPlaceDf = horse_race_data100k[horse_race_data100k['Average Odds']<=averageFourthPlaceOdds]

In [218]:
#output of new dataframe
fourthPlaceDf.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds
2,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2,9.679583
6,VH5mOjX8,22364,11.20,6.0,108.6571,2021-06-01,1800,11.2,9.915833
8,VH5mOjX8,23089,8.82,5.0,108.4495,2021-06-01,1800,11.2,8.509186
18,8gzdZLq5,13001,8.86,12.0,109.1853,2021-06-01,1800,10.0,8.258444
19,8gzdZLq5,13012,8.68,4.0,104.8865,2021-06-01,1800,10.0,9.310526


In [219]:
#resetting the indexes as they got all funky above
fourthPlaceDf = fourthPlaceDf.reset_index()

In [220]:
fourthPlaceDf.head()

,index,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds
0,2,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2,9.679583
1,6,VH5mOjX8,22364,11.20,6.0,108.6571,2021-06-01,1800,11.2,9.915833
2,8,VH5mOjX8,23089,8.82,5.0,108.4495,2021-06-01,1800,11.2,8.509186
3,18,8gzdZLq5,13001,8.86,12.0,109.1853,2021-06-01,1800,10.0,8.258444
4,19,8gzdZLq5,13012,8.68,4.0,104.8865,2021-06-01,1800,10.0,9.310526


In [221]:
fourthPlaceDf = fourthPlaceDf.drop(['index'], axis = 1)

In [222]:
fourthPlaceDf.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds
0,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2,9.679583
1,VH5mOjX8,22364,11.20,6.0,108.6571,2021-06-01,1800,11.2,9.915833
2,VH5mOjX8,23089,8.82,5.0,108.4495,2021-06-01,1800,11.2,8.509186
3,8gzdZLq5,13001,8.86,12.0,109.1853,2021-06-01,1800,10.0,8.258444
4,8gzdZLq5,13012,8.68,4.0,104.8865,2021-06-01,1800,10.0,9.310526


In [223]:
fourthPlaceDf['horse_time'].fillna(value = fourthPlaceDf['horse_time'].mean(), inplace = True)

In [224]:
#checking to see how many records remain
fourthPlaceDf.shape[0]

17180

In [225]:
#populating a dictionary with each horse and all of its finish times
horseTimeMap = {}
for row in range(fourthPlaceDf.shape[0]):
    if str(fourthPlaceDf.loc[row,'horse_id']) in horseTimeMap: #if horse id is already in the dictionary
        horseTimeMap[str(fourthPlaceDf.loc[row,'horse_id'])].append(fourthPlaceDf.loc[row,'horse_time']) #append to value
    else:
        horseTimeMap[str(fourthPlaceDf.loc[row,'horse_id'])] = [fourthPlaceDf.loc[row,'horse_time']] #create new key and value

In [226]:
horseTimeMap['10133']

[108.8827,
 107.9304,
 108.3212,
 109.3551,
 107.813,
 105.4186,
 106.9227,
 107.0871,
 107.7232,
 109.8438,
 109.6449,
 108.4013,
 102.9318,
 109.067,
 107.7928702793692,
 108.0933,
 107.1399,
 110.5893,
 108.1235,
 109.6187,
 106.736,
 106.3112,
 110.1966,
 109.0724,
 108.0883,
 105.3227,
 109.9632,
 106.3916,
 106.8119,
 107.8863,
 109.0523,
 109.6232,
 107.8506,
 107.5027,
 108.2742,
 102.9987,
 105.4932,
 107.4425,
 108.0057,
 108.8214,
 109.4842,
 104.8136,
 109.0722,
 108.1572,
 110.0585,
 108.7463,
 109.9109,
 109.8646]

In [227]:
#now averaging each horses values from the horseTimeMap Dictionary and storing in new dictionary mapped to horse
horseTimeAverageMap = {}
for key,val in horseTimeMap.items():
    if key not in horseTimeAverageMap:
        horseTimeAverageMap[key] = sum(val)/len(val)

In [228]:
horseTimeAverageMap['10133']

107.97192854748685

In [229]:
#adding in a the average race time column to fourth place df
fourthPlaceDf['Average Race Time'] = ""

In [230]:
#populating the column
for row in range(fourthPlaceDf.shape[0]):
    fourthPlaceDf.loc[row,'Average Race Time'] = horseTimeAverageMap[str(fourthPlaceDf.loc[row,'horse_id'])]

In [231]:
#output of above
fourthPlaceDf.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds,Average Race Time
0,VH5mOjX8,10133,10.51,10.0,108.8827,2021-06-01,1800,11.2,9.679583,107.971929
1,VH5mOjX8,22364,11.20,6.0,108.6571,2021-06-01,1800,11.2,9.915833,107.586539
2,VH5mOjX8,23089,8.82,5.0,108.4495,2021-06-01,1800,11.2,8.509186,107.658318
3,8gzdZLq5,13001,8.86,12.0,109.1853,2021-06-01,1800,10.0,8.258444,107.905472
4,8gzdZLq5,13012,8.68,4.0,104.8865,2021-06-01,1800,10.0,9.310526,108.235968


In [232]:
#using the standard deviation of 4th row because in speed stats that is the record
#as of now am manually inputting the 4, to optimize need to have it so it looks up by the distance
standard_deviation = speed_stats.loc[4,'Standard Deviation']
median_race_time = speed_stats.loc[4,'Mean']
desired_time = median_race_time - 1*standard_deviation
print(standard_deviation, median_race_time)

1.71 107.8


In [233]:
#now creating new dataframe where each horse race time is less than or equal to median race time - 1
 
fourthPlaceMedian = fourthPlaceDf[fourthPlaceDf['Average Race Time']<= desired_time]

In [234]:
fourthPlaceMedian = fourthPlaceMedian.reset_index()


In [235]:
fourthPlaceMedian = fourthPlaceMedian.drop(['index'],axis = 1)
fourthPlaceMedian.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds,Average Race Time
0,QXrq7dag,389,10.07,8.0,108.4904,2021-06-02,1800,10.31,8.886667,104.888017
1,4VXwOOj0,10123,5.14,10.0,105.5698,2021-06-02,1800,9.97,8.5325,105.95185
2,4VXwOOj0,10712,6.83,5.0,104.8307,2021-06-02,1800,9.97,6.83,104.8307
3,KLNixLlW,2428,9.87,1.0,105.9150,2021-06-03,1800,9.13,9.87,105.915
4,CDngvl5u,19728,9.73,4.0,109.7456,2021-06-04,1800,9.93,9.727143,105.440057


In [236]:
#remaining records left after slice
fourthPlaceMedian.shape[0]

222

In [237]:
#must place 1st, 2nd, 11th, or 12th 40% of the time

In [238]:
horsePlacementDict = {} #dictionary to hold all placements for each horse
horsePercentPlaceDict = {} #dicionary to hold each horses percent of finishing 1st 2nd 11th 12th
def fortyPercentPlace(values): #function to check if placement is 1st 2nd 11th 12th
    forty = [] #list to hold values that are 1,2,11,12
    for placement in values:
        if placement in [1.0,2.0,11.0,12.0]: #if placement is 1 2 11 or 12, then put in list
            forty.append(placement)
    return len(forty)/len(values)        

In [239]:
for row in range(fourthPlaceMedian.shape[0]):
    if fourthPlaceMedian.loc[row,'horse_id'] in horsePlacementDict: #if horse in dictionary, append its placement value
        horsePlacementDict[fourthPlaceMedian.loc[row,'horse_id']].append(fourthPlaceMedian.loc[row,'place'])
    else:
        horsePlacementDict[fourthPlaceMedian.loc[row,'horse_id']] = [fourthPlaceMedian.loc[row,'place']] #if not, add horse and placement value

In [240]:
horsePlacementDict[389] #test

[8.0, 3.0, 1.0, 3.0, 1.0, 2.0]

In [241]:
for key,val in horsePlacementDict.items(): #place each horseid and then pass in its list of placements to the function
    if key not in horsePercentPlaceDict:
        horsePercentPlaceDict[key] = fortyPercentPlace(val) #function call form three cells above

In [242]:
horsePercentPlaceDict[389]

0.5

In [243]:
fourthPlaceMedian['Percent first, Second, Last'] = ""


In [244]:
for row in range(fourthPlaceMedian.shape[0]):
    fourthPlaceMedian.loc[row,'Percent first, Second, Last'] = horsePercentPlaceDict[fourthPlaceMedian.loc[row,'horse_id']]

In [245]:
fourthPlaceMedian.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds,Average Race Time,"Percent first, Second, Last"
0,QXrq7dag,389,10.07,8.0,108.4904,2021-06-02,1800,10.31,8.886667,104.888017,0.5
1,4VXwOOj0,10123,5.14,10.0,105.5698,2021-06-02,1800,9.97,8.5325,105.95185,0.25
2,4VXwOOj0,10712,6.83,5.0,104.8307,2021-06-02,1800,9.97,6.83,104.8307,0.0
3,KLNixLlW,2428,9.87,1.0,105.9150,2021-06-03,1800,9.13,9.87,105.915,1.0
4,CDngvl5u,19728,9.73,4.0,109.7456,2021-06-04,1800,9.93,9.727143,105.440057,0.285714


In [246]:
final_race_list = fourthPlaceMedian[fourthPlaceMedian['Percent first, Second, Last'] >= .4] 

In [247]:
final_race_list.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds,Average Race Time,"Percent first, Second, Last"
0,QXrq7dag,389,10.07,8.0,108.4904,2021-06-02,1800,10.31,8.886667,104.888017,0.5
3,KLNixLlW,2428,9.87,1.0,105.9150,2021-06-03,1800,9.13,9.87,105.915,1.0
6,adhbQe7V,325,7.37,11.0,107.0876,2021-06-11,1800,10.55,7.735,106.065488,0.625
8,56CErNaS,1210,8.56,1.0,102.8329,2021-06-12,1800,9.04,8.56,102.8329,1.0
9,BtWwbIEn,296,7.73,3.0,105.8776,2021-06-12,1800,9.32,7.487273,105.869745,0.636364


In [248]:
final_race_list = final_race_list.reset_index()

In [249]:
final_race_list = final_race_list.drop(['index'], axis = 1)

In [250]:
final_race_list.head()

,race_id,horse_id,odds,place,horse_time,start_time,distance,Fourth Place Odds,Average Odds,Average Race Time,"Percent first, Second, Last"
0,QXrq7dag,389,10.07,8.0,108.4904,2021-06-02,1800,10.31,8.886667,104.888017,0.5
1,KLNixLlW,2428,9.87,1.0,105.9150,2021-06-03,1800,9.13,9.87,105.915,1.0
2,adhbQe7V,325,7.37,11.0,107.0876,2021-06-11,1800,10.55,7.735,106.065488,0.625
3,56CErNaS,1210,8.56,1.0,102.8329,2021-06-12,1800,9.04,8.56,102.8329,1.0
4,BtWwbIEn,296,7.73,3.0,105.8776,2021-06-12,1800,9.32,7.487273,105.869745,0.636364


In [251]:
listOfHorses = []
for row in range(final_race_list.shape[0]):
    if final_race_list.loc[row,'horse_id'] not in listOfHorses:
        listOfHorses.append(final_race_list.loc[row,'horse_id'])

In [252]:
len(listOfHorses)

80

In [253]:
final_race_list.shape[0]

133

In [254]:
horse_data.shape[0]

49272

In [255]:
horse_data.head() #reobserving the horse dataset

,horse_id,name,genotype,bloodline,breed_type,color,super_coat,mother,father,horse_type,birthday
0,0,Thin Frivolity,Z1,Nakamoto,genesis,Hairy Canary,f,NaN,NaN,Colt,2019-04-04 01:39:07
1,1,Chase Jackson,Z1,Nakamoto,genesis,Cornsilk,f,NaN,NaN,Mare,2019-01-17 00:21:42
2,2,Wall Street,Z1,Nakamoto,genesis,Bittersweet Shimmer,f,NaN,NaN,Stallion,2019-01-17 00:32:57
3,3,Millions,Z1,Nakamoto,genesis,Fandango,f,NaN,NaN,Filly,2019-01-17 02:44:23
4,4,Guns and Horses,Z1,Nakamoto,genesis,White Smoke,f,NaN,NaN,Colt,2019-01-18 06:44:24


In [256]:
horse_data['Gem?'] = "" #adding in column to denote gem

In [257]:
for row in range(horse_data.shape[0]): #populating column if the horse id is in the listOfHorses
    if horse_data.loc[row,'horse_id'] in listOfHorses:
        horse_data.loc[row,'Gem?'] = "Gem"

In [258]:
gem_Horses = horse_data[horse_data['Gem?'] == "Gem"]

In [259]:
gem_Horses.head(80)

,horse_id,name,genotype,bloodline,breed_type,color,super_coat,mother,father,horse_type,birthday,Gem?
296,296,Proud Delight,Z1,Nakamoto,genesis,Rosy Brown,f,NaN,NaN,Stallion,2019-03-03 00:12:40,Gem
325,325,Barok,Z1,Nakamoto,genesis,Cinereous,f,NaN,NaN,Mare,2019-03-05 12:04:32,Gem
329,329,Ace of Spades,Z1,Nakamoto,genesis,Azure Mist,f,NaN,NaN,Mare,2019-03-05 12:07:43,Gem
389,389,Bruiser,Z1,Nakamoto,genesis,Arylide Yellow,f,NaN,NaN,Mare,2019-03-08 07:20:15,Gem
753,753,Too Proud,Z2,Nakamoto,genesis,Hairy Canary,f,NaN,NaN,Stallion,2019-03-22 06:18:11,Gem
...,...,...,...,...,...,...,...,...,...,...,...,...
46154,46154,So Many People,Z10,Buterin,genesis,Desert Sand,f,NaN,NaN,Colt,2021-07-10 07:57:52,Gem
46472,46472,Used Launder Water,Z10,Buterin,genesis,Firebrick,f,NaN,NaN,Mare,2021-07-10 08:23:52,Gem
47485,47485,Mack Attack,Z3,Nakamoto,legendary,Atomic Tangerine,f,13421.0,424.0,Colt,2021-07-10 09:59:07,Gem
47636,47636,Kid Pony,Z34,Buterin,elite,Burnt Umber,f,17777.0,21972.0,Colt,2021-07-10 15:18:36,Gem


In [260]:
listOfHorses

[389,
 2428,
 325,
 1210,
 296,
 9521,
 13553,
 27493,
 37879,
 37233,
 32289,
 18461,
 26072,
 40554,
 46154,
 47485,
 40150,
 43414,
 44215,
 31570,
 18431,
 42996,
 31642,
 34132,
 47786,
 44222,
 25382,
 22920,
 10361,
 23056,
 30941,
 34992,
 35567,
 36820,
 31941,
 26070,
 18633,
 36848,
 29155,
 39025,
 9461,
 39425,
 35520,
 42330,
 47636,
 43409,
 46472,
 4316,
 23223,
 21408,
 27310,
 25886,
 14557,
 10608,
 33919,
 329,
 753,
 39863,
 44566,
 45968,
 45522,
 9622,
 2776,
 15018,
 21139,
 20687,
 31410,
 38485,
 34439,
 26563,
 13919,
 37697,
 40347,
 38998,
 39200,
 39835,
 45601,
 45101,
 44253,
 39678]

In [261]:
gem_Horses.to_csv('Gems.csv', index=False)